In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
from iac_sketch import data, etl, sketch, transform, system_tests

In [ ]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(etl)
importlib.reload(sketch)
importlib.reload(transform)
importlib.reload(system_tests)
# architect = sketch.Architect("../public/components")
# registry = architect.parse()

In [ ]:
architect = sketch.Architect()
registry = architect.perform_registry_etl()
tests, test_results = architect.validate_registry()
tests

In [ ]:
with pd.option_context('display.max_colwidth', None):
    print(tests.query("errors.notna() and errors != ''")["errors"])

In [ ]:
# Show results
for test_key, test_result in test_results.items():
    print(f"{test_key}:")
    display(test_result)
    print("")

In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry.entities)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)